# Dynamic Example Helpers

> Functions to generate dynamic examples for help text:

In [ ]:
#| default_exp cli.dynamic_examples

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Tuple, Any, Optional
from cjm_fasthtml_tailwind.cli.utils import (
    SearchResult, discover_utility_modules,
    load_code_from_file, display_items_generic, handle_module_not_found,
    find_usage_in_items, get_view_command, format_usage_examples, simple_item_formatter,
    indented_item_formatter, list_utility_modules
)
from cjm_fasthtml_tailwind.cli.factory_extraction import (
    list_all_factories, list_module_factories, get_factory_by_name
)
from cjm_fasthtml_tailwind.cli.example_discovery import (
    list_all_examples, get_example_by_name, list_module_examples
)
from cjm_fasthtml_tailwind.cli.helper_discovery import get_module_helpers
from cjm_fasthtml_tailwind.cli.search import search_all
from cjm_fasthtml_tailwind.cli.core_utils_discovery import get_core_utilities
from cjm_fasthtml_tailwind.cli.test_code import execute_test_code
from cjm_fasthtml_tailwind.cli.imports import get_recommended_imports
from cjm_fasthtml_tailwind.cli.pattern_scanner import (
    analyze_and_suggest_input, display_input_analysis, 
    InputType, detect_input_type
)
from cjm_fasthtml_tailwind.cli.cli_config import LibraryConfig, get_active_config

from cjm_fasthtml_tailwind.cli.display import (
    display_modules, display_module_factories, display_all_factories, 
    display_module_examples, display_all_examples, display_example_source, 
    display_module_helpers, display_helper_source,
    display_all_helpers, display_factory_info, display_search_results, 
    display_core_utility_source, display_core_utilities, display_imports, display_test_code_result
)

In [ ]:
#| export
def get_example_modules(
    limit: int = 2,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
) -> str:  # TODO: Add return description
    """Get example module names dynamically."""
    if config is None:
        config = get_active_config()
        
    modules = list_utility_modules(config)
    if not modules:
        return ""
    
    module_names = list(modules.keys())[:limit]
    return ", ".join(module_names)

In [ ]:
#| export
def get_example_factories(
    module_name: str = None,  # TODO: Add description
    limit: int = 4,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
) -> str:  # TODO: Add return description
    """Get example factory names dynamically."""
    if config is None:
        config = get_active_config()
        
    if module_name:
        factories = list_module_factories(module_name)
    else:
        # Get from first available module
        all_factories = list_all_factories()
        if not all_factories:
            return ""
        
        first_module = list(all_factories.keys())[0]
        factories = all_factories[first_module]
    
    if not factories:
        return ""
    
    factory_names = [f.name for f in factories[:limit]]
    return ", ".join(factory_names)

In [ ]:
#| export
def get_example_features(
    module_name: str = None,  # TODO: Add description
    limit: int = 3,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
) -> str:  # TODO: Add return description
    """Get example feature names dynamically."""
    if config is None:
        config = get_active_config()
        
    if module_name:
        examples = list_module_examples(module_name)
    else:
        # Get from first module with examples
        all_examples = list_all_examples()
        if not all_examples:
            return ""
        
        first_module = list(all_examples.keys())[0]
        examples = all_examples[first_module]
    
    if not examples:
        return ""
    
    feature_names = [e.feature for e in examples[:limit]]
    return ", ".join(feature_names)

In [ ]:
#| export
def get_example_helpers(
    module_name: str = None,  # TODO: Add description
    limit: int = 2,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
) -> str:  # TODO: Add return description
    """Get example helper function names dynamically."""
    if config is None:
        config = get_active_config()
        
    if module_name:
        helpers = get_module_helpers(module_name)
    else:
        # Get from first module with helpers
        for mod_name, module in discover_utility_modules(config):
            helpers = get_module_helpers(mod_name)
            if helpers:
                break
        else:
            return ""
    
    if not helpers:
        return ""
    
    helper_names = [h.name for h in helpers[:limit]]
    return ", ".join(helper_names)

In [ ]:
#| export
def get_example_core_utils(
    limit: int = 2,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
) -> str:  # TODO: Add return description
    """Get example core utility names dynamically."""
    if config is None:
        config = get_active_config()
        
    utilities = get_core_utilities(config)
    if not utilities:
        return ""
    
    util_names = [u.name for u in utilities[:limit]]
    return ", ".join(util_names)

In [ ]:
#| export
def get_combine_classes_example(
) -> str:  # TODO: Add return description
    """Get a dynamic example of combine_classes usage."""
    # Try to find combine_classes usage in examples
    from cjm_fasthtml_tailwind.cli.example_discovery import list_all_examples
    from cjm_fasthtml_tailwind.cli.search import search_all
    
    # Search for combine_classes usage
    results = search_all('combine_classes', include_source=True, content_types=['examples'])
    
    if results:
        # Look for a simple combine_classes call
        import re
        for result in results:
            # Try to extract a combine_classes call from the match context
            match = re.search(r'combine_classes\([^)]+\)', result.match_context)
            if match:
                example = match.group(0)
                # Make sure it's not too long
                if len(example) <= 50:
                    return f"print({example})"
    
    # If no example found, create a minimal one with discovered factories
    factories = get_example_factories(limit=2).split(', ')
    if len(factories) >= 2:
        return f"print(combine_classes({factories[0]}(4), {factories[1]}.auto))"
    
    return "print(combine_classes(...))"

In [ ]:
#| export
def get_example_test_code(
    limit_chars: int = 50  # TODO: Add description
) -> str:  # TODO: Add return description
    """Get an actual code example from test functions."""
    # Try to get examples from the first available module
    all_examples = list_all_examples()
    if not all_examples:
        return "<code-here>"
    
    # Get the first module with examples
    first_module_name = list(all_examples.keys())[0]
    examples = all_examples[first_module_name]
    
    if not examples:
        return "<code-here>"
    
    # Look for a basic example first
    for example in examples:
        if 'basic' in example.feature:
            # Extract a simple assertion from the source
            lines = example.source.strip().split('\n')
            for line in lines:
                line = line.strip()
                # Look for simple assert statements
                if line.startswith('assert') and 'str(' in line and '==' in line:
                    # Extract the expression being tested
                    parts = line.split('==')
                    if len(parts) == 2:
                        expr = parts[0].replace('assert', '').strip()
                        if len(expr) <= limit_chars:
                            return f"print({expr})"
    
    # If no basic example, try to find any simple assertion
    for example in examples:
        lines = example.source.strip().split('\n')
        for line in lines:
            line = line.strip()
            if line.startswith('assert') and 'str(' in line and '==' in line:
                parts = line.split('==')
                if len(parts) == 2:
                    expr = parts[0].replace('assert', '').strip()
                    if len(expr) <= limit_chars:
                        return f"print({expr})"
    
    # Last resort: try to find any factory usage
    for example in examples:
        import re
        # Look for factory calls like p(4), m.auto, w.full
        matches = re.findall(r'\b([a-z]+(?:\.[a-z_]+)?(?:\([^)]+\))?)\b', example.source)
        for match in matches:
            if '(' in match or '.' in match:
                if len(match) <= 20:  # Keep it short
                    return f"print(str({match}))"
    
    return "<code-here>"

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()